# Train and deploy the model

Training to deploy a model, and training a model with Python SDK

---

## Import and everything

In [2]:
import sys
!{sys.executable} -m pip install sagemaker-experiments

In [3]:
import sagemaker

sess = sagemaker.Session()
bucket = "s3://test-sagemaker-examples-1357942113492"
prefix = "/Exp_2/"

train_loc = "Train/Wine-Quality-2023-05-22T11-12-10/part-00000-5812d1e3-d6a0-4cc1-afd8-1f22f194c20b-c000.csv"
test_loc = "Test/Wine-Quality-2023-05-22T11-12-10/part-00000-272ca3b5-c289-4a25-ad9d-cfd7e98bd977-c000.csv"
validation_loc = "Validation/Wine-Quality-2023-05-22T11-12-10/part-00000-61ee30c8-9b7c-4e79-bad1-1277435f4268-c000.csv"

role = sagemaker.get_execution_role()

In [4]:
import os
import pandas as pd
import numpy as np
import boto3
import time

from sagemaker.predictor import CSVSerializer
from sagemaker.inputs import TrainingInput

from time import strftime
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker
from botocore.exceptions import ClientError

## Test to read CSV

In [5]:
test = pd.read_csv(bucket + prefix + test_loc)
test

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,If_White_Then_1
0,3.0,7.0,0.270,0.36,20.7,0.045,45,170,1.00100,3.00,0.45,8.8,1
1,2.0,8.6,0.230,0.40,4.2,0.035,17,109,0.99470,3.14,0.53,9.7,1
2,5.0,6.2,0.660,0.48,1.2,0.029,29,75,0.98920,3.33,0.39,12.8,1
3,3.0,7.4,0.340,0.42,1.1,0.033,17,171,0.99170,3.12,0.53,11.3,1
4,4.0,7.2,0.320,0.36,2.0,0.033,37,114,0.99060,3.10,0.71,12.3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,3.0,6.7,0.670,0.02,1.9,0.061,26,42,0.99489,3.39,0.82,10.9,0
1256,3.0,6.7,0.160,0.64,2.1,0.059,24,52,0.99494,3.34,0.71,11.2,0
1257,2.0,6.2,0.560,0.09,1.7,0.053,24,32,0.99402,3.54,0.60,11.3,0
1258,2.0,6.1,0.715,0.10,2.6,0.053,13,27,0.99362,3.57,0.50,11.9,0


In [6]:
train = pd.read_csv(bucket + prefix + train_loc)
train

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,If_White_Then_1
0,3.0,7.0,0.270,0.36,20.7,0.045,45,170,1.00100,3.00,0.45,8.8,1
1,3.0,6.3,0.300,0.34,1.6,0.049,14,132,0.99400,3.30,0.49,9.5,1
2,3.0,8.1,0.280,0.40,6.9,0.050,30,97,0.99510,3.26,0.44,10.1,1
3,3.0,7.2,0.230,0.32,8.5,0.058,47,186,0.99560,3.19,0.40,9.9,1
4,3.0,7.2,0.230,0.32,8.5,0.058,47,186,0.99560,3.19,0.40,9.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4031,2.0,6.2,0.600,0.08,2.0,0.090,32,44,0.99490,3.45,0.58,10.5,0
4032,3.0,5.9,0.550,0.10,2.2,0.062,39,51,0.99512,3.52,0.76,11.2,0
4033,3.0,6.3,0.510,0.13,2.3,0.076,29,40,0.99574,3.42,0.75,11.0,0
4034,2.0,5.9,0.645,0.12,2.0,0.075,32,44,0.99547,3.57,0.71,10.2,0


## Create Experiments and Trials

In [7]:
create_date = strftime("%Y-%m-%d-%H-%M-%S")
exp_name = "Wine-Quality-Experiment"
exp_desc = "Practise Project"

try:
    experiment = Experiment.create(experiment_name=exp_name.format(create_date), 
                                   description=exp_desc)
except ClientError as e:
    print(f'{exp_name} already exists and will be reused.')

Wine-Quality-Experiment already exists and will be reused.


In [8]:
trial_name = "Wine-Quality-Trial-15"

demo_trial = Trial.create(trial_name = trial_name.format(create_date),
                          experiment_name = exp_name)

## Train

In [9]:
s3_input_train = TrainingInput(
    s3_data=bucket + prefix + train_loc, content_type="csv",
)
s3_input_validation = TrainingInput(
    s3_data=bucket + prefix + validation_loc, content_type="csv",
)

In [10]:
container = sagemaker.image_uris.retrieve("xgboost", sess.boto_region_name, '1.5-1')

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [11]:
exp_config = {"ExperimentName": exp_name,
              "TrialName": trial_name,
              "TrialComponentDisplayName": "TrainingJob"}

In [12]:
hyper_par = {
    "max_depth":"5",
    "eta": "0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample": "0.8",
    "objective": "multi:softmax",
    "eval_metric": "merror",
    "num_round":"100",
    "num_class":"10",}

output_path = bucket + prefix + "Output"

xgb = sagemaker.estimator.Estimator(image_uri=container,
                                    hyperparameters=hyper_par,
                                    role=role,
                                    instance_count=1,
                                    instance_type="ml.m4.xlarge",
                                    output_path=output_path,
                                    sagemaker_session=sess)

In [13]:
# xgb.fit({"train": s3_input_train, "validation": s3_input_validation}, experiment_config=exp_config)

In [14]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

n_jobs = 50
n_parallel_jobs = 3

hpt_ranges = {
    'eta': ContinuousParameter(0.1, .5),
    'min_child_weight': ContinuousParameter(0., 10.),
    'max_depth': IntegerParameter(1, 10),
    'gamma': IntegerParameter(2,8),
    'subsample': ContinuousParameter(0.1, 1.)
}


tuner_parameters = {
    'estimator': xgb,
    'base_tuning_job_name': 'bayesian',                   
    'objective_metric_name': 'validation:accuracy',
    'objective_type': 'Maximize',
    'hyperparameter_ranges': hpt_ranges,
    'strategy': 'Bayesian',
    'max_jobs': n_jobs,
    'max_parallel_jobs': n_parallel_jobs
}


In [16]:
tuner = HyperparameterTuner(**tuner_parameters)
tuner.fit({"train": s3_input_train, "validation": s3_input_validation}, wait=False)
tuner_name = tuner.describe()['HyperParameterTuningJobName']
print(f'Tuning job in progress: {tuner_name}')

INFO:sagemaker:Creating hyperparameter tuning job with name: bayesian-230522-0908


Tuning job in progress: bayesian-230522-0908


In [17]:
tuner.wait()

..............................................................................................................................................................................................................................................!


In [18]:
sagemaker.HyperparameterTuningJobAnalytics(tuner_name).dataframe()[:10]

,eta,gamma,max_depth,min_child_weight,subsample,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,0.274470,2.0,10.0,8.796005,0.700940,bayesian-230522-0908-050-0f8b70fb,Completed,0.63726,2023-05-22 09:27:56+00:00,2023-05-22 09:28:33+00:00,37.0
1,0.253748,2.0,10.0,6.544147,0.801938,bayesian-230522-0908-049-a2e180ce,Completed,0.63627,2023-05-22 09:27:49+00:00,2023-05-22 09:28:31+00:00,42.0
2,0.246704,2.0,10.0,1.455623,0.755248,bayesian-230522-0908-048-4c3e1ff8,Completed,0.63925,2023-05-22 09:27:29+00:00,2023-05-22 09:28:11+00:00,42.0
3,0.246586,2.0,10.0,4.698270,0.779389,bayesian-230522-0908-047-de03cab5,Completed,0.64321,2023-05-22 09:26:56+00:00,2023-05-22 09:27:33+00:00,37.0
4,0.262810,2.0,10.0,2.259974,0.791495,bayesian-230522-0908-046-6b149df1,Completed,0.65015,2023-05-22 09:26:48+00:00,2023-05-22 09:27:30+00:00,42.0
5,0.230460,2.0,10.0,1.660058,0.765628,bayesian-230522-0908-045-bc25a601,Completed,0.64717,2023-05-22 09:26:35+00:00,2023-05-22 09:27:12+00:00,37.0
6,0.220678,2.0,10.0,0.000000,0.583747,bayesian-230522-0908-044-598c569f,Completed,0.63528,2023-05-22 09:25:55+00:00,2023-05-22 09:26:37+00:00,42.0
7,0.100000,2.0,10.0,5.346183,1.000000,bayesian-230522-0908-043-2f426854,Completed,0.64123,2023-05-22 09:25:45+00:00,2023-05-22 09:26:22+00:00,37.0
8,0.100000,2.0,10.0,10.000000,0.694352,bayesian-230522-0908-042-69a84039,Completed,0.63528,2023-05-22 09:25:44+00:00,2023-05-22 09:26:21+00:00,37.0
9,0.304483,2.0,10.0,0.000000,0.445466,bayesian-230522-0908-041-f937e28d,Completed,0.64916,2023-05-22 09:25:05+00:00,2023-05-22 09:25:41+00:00,36.0


In [21]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuner_name)['BestTrainingJob']

{'TrainingJobName': 'bayesian-230522-0908-033-70fd685c',
 'TrainingJobArn': 'arn:aws:sagemaker:eu-west-1:790592228004:training-job/bayesian-230522-0908-033-70fd685c',
 'CreationTime': datetime.datetime(2023, 5, 22, 9, 22, 40, tzinfo=tzlocal()),
 'TrainingStartTime': datetime.datetime(2023, 5, 22, 9, 22, 50, tzinfo=tzlocal()),
 'TrainingEndTime': datetime.datetime(2023, 5, 22, 9, 23, 32, tzinfo=tzlocal()),
 'TrainingJobStatus': 'Completed',
 'TunedHyperParameters': {'eta': '0.3045733696136262',
  'gamma': '2',
  'max_depth': '10',
  'min_child_weight': '0.0',
  'subsample': '0.7602335489240319'},
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:accuracy',
  'Value': 0.6590700149536133},
 'ObjectiveStatus': 'Succeeded'}

In [22]:
!pip install -Uq pip altair

In [30]:
from scripts.visualization import visualize_tuning_job
visualize_tuning_job(tuner, trials_only=True)

Tuning job bayesian-230522-0908      status: Completed

Number of training jobs with valid objective: 50
Lowest: 0.5550000071525574 Highest 0.6590700149536133


,eta,gamma,max_depth,min_child_weight,subsample,TrainingJobName,TrainingJobStatus,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds,TuningJobName,validation:accuracy
17,0.304573,2.0,10.0,0.000000,0.760234,bayesian-230522-0908-033-70fd685c,Completed,2023-05-22 09:22:50+00:00,2023-05-22 09:23:32+00:00,42.0,bayesian-230522-0908,0.65907
4,0.262810,2.0,10.0,2.259974,0.791495,bayesian-230522-0908-046-6b149df1,Completed,2023-05-22 09:26:48+00:00,2023-05-22 09:27:30+00:00,42.0,bayesian-230522-0908,0.65015
9,0.304483,2.0,10.0,0.000000,0.445466,bayesian-230522-0908-041-f937e28d,Completed,2023-05-22 09:25:05+00:00,2023-05-22 09:25:41+00:00,36.0,bayesian-230522-0908,0.64916
5,0.230460,2.0,10.0,1.660058,0.765628,bayesian-230522-0908-045-bc25a601,Completed,2023-05-22 09:26:35+00:00,2023-05-22 09:27:12+00:00,37.0,bayesian-230522-0908,0.64717
11,0.331222,2.0,10.0,1.570376,0.774195,bayesian-230522-0908-039-fde711fe,Completed,2023-05-22 09:24:44+00:00,2023-05-22 09:25:26+00:00,42.0,bayesian-230522-0908,0.64618
16,0.400020,2.0,10.0,0.000000,0.773536,bayesian-230522-0908-034-503c69f2,Completed,2023-05-22 09:22:48+00:00,2023-05-22 09:23:29+00:00,41.0,bayesian-230522-0908,0.64618
3,0.246586,2.0,10.0,4.698270,0.779389,bayesian-230522-0908-047-de03cab5,Completed,2023-05-22 09:26:56+00:00,2023-05-22 09:27:33+00:00,37.0,bayesian-230522-0908,0.64321
19,0.346615,2.0,10.0,4.309399,0.787115,bayesian-230522-0908-031-f5c1fc7c,Completed,2023-05-22 09:21:51+00:00,2023-05-22 09:22:28+00:00,37.0,bayesian-230522-0908,0.64321
12,0.137121,2.0,10.0,0.000000,0.857007,bayesian-230522-0908-038-0f011f8c,Completed,2023-05-22 09:24:05+00:00,2023-05-22 09:24:47+00:00,42.0,bayesian-230522-0908,0.64321
7,0.100000,2.0,10.0,5.346183,1.000000,bayesian-230522-0908-043-2f426854,Completed,2023-05-22 09:25:45+00:00,2023-05-22 09:26:22+00:00,37.0,bayesian-230522-0908,0.64123


ValueError: "accuracy" is not one of the valid encoding data types: O, N, Q, T, G.
For more details, see https://altair-viz.github.io/user_guide/encodings/index.html#encoding-data-types. If you are trying to use a column name that contains a colon, prefix it with a backslash; for example "column\:name" instead of "column:name".

alt.VConcatChart(...)